# Capstone Project Modul 2 - New York City TLC Data Analysis
Faishal Hananta Aji JCDSBSDAM-31

# **1. Latar Belakang**


New York City Taxi & Limousine Commission merupakan sebuah perusahaan yang mengatur industri antar-jemput mobil (dari taksi kota dan limousine hingga layanan antar-jemput berbasis app seperti Uber dan Lyft) yang beroperasi di kota New York dari segi lisensi hingga regulasi kegiatan sehari-hari.

## **1.1. Pernyataan Masalah**


Perusahaan New York City TLC ingin mengetahui bagaimana kegiatan sehari-hari armada TLC dapat dioptimalkan untuk memaksimalisasi efisiensi kerja armada serta profit yang didapatkan oleh perusahaan. 

Sebagai sebuah data analyst, kita mencoba untuk menjawab pertanyaan berikut:

Bagaimana mengoptimalkan struktur tarif, jarak, dan waktu tempuh untuk meningkatkan efisiensi dan profitabilitas layanan taksi?

# **2. Data**


Untuk menjawab pertanyaan di atas, kita dapat menganalisis data catatan pemesanan yang dipegang oleh New York City TLC. Data yang digunakan merupakan data New York City TLC selama bulan Januari pada tahun 2023.

In [1335]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import normaltest, chi2_contingency, mannwhitneyu, ttest_ind

import warnings
warnings.filterwarnings("ignore")

Dataset yang digunakan dalam analysis ini merupakan data informasi dari tiap transaksi jasa layanan antar-jemput taksi, limusin dan mobil yang dikelola oleh New York City TLC. Menurut data dictionary yang telah diberikan oleh pihak New York City TLC, terdapat 18 jenis data yang disimpan oleh dataset ini, yaitu:

- VendorID: 
    - Provider LPEP (Street Hail Livery Technology System) yang memberikan data tersebut ke New York TLC
- lpep_pickup_datetime: 
    - Waktu dan tanggal taksimeter taksi/limusin/mobil dinyalakan ketika perjalanan dimulai dalam sebuah transaksi antar-jemput
- lpep_dropoff_datetime
    - Waktu dan tanggal taksimeter taksi/limusin/mobil dimatikan ketika perjalanan selesai dalam sebuah transaksi antar-jemput
- Passenger_count
    - Jumlah penumpang yang diangkut taksi/limusin/mobil dalam sebuah transaksi antar-jemput (diisi oleh driver)
- Trip_distance
    - Panjang perjalanan dalam sebuah transaksi antar-jemput, sebagaimana tercatat pada taksimeter
- PULocationID
    - ID yang menggambarkan lokasi transaksi antar-jemput dimulai
- DOLocationID
    - ID yang menggambarkan lokasi transaksi antar-jemput selesai
- RateCodeID
    - ID yang menggambarkan tipe fare-rate yang menentukan harga akhir yang perlu dibayar setelah akhir transaksi antar-jemput
- Store_and_fwd_flag
    - Label yang menggambarkan apakah data perjalanan dari sebuah transaksi antar-jemput disimpan terlebih dahulu di taksi/limusin/mobil sebelum dikirimkan ke vendor karena terjadi masalah koneksi di tengah perjalanan
- Payment_type
    - ID yang menggambarkan metode pembayaran yang digunakan oleh penumpang 
- Fare_amount
    - Jumlah fare taksi/limosin/mobil, yang ditentukan dari panjang perjalanan yang terlihat pada taksimeter
- MTA_tax
    - Pajak MTA yang ditentukan oleh rate
- Improvement_surcharge
    - Biaya tambahan yang dipungut demi perawatan dan pengembangan layanan antar-jemput New York TLC 
- Tip_amount
    - Jumlah tip yang diterima driver. Tip yang tercatat hanyalah tip yang dikirimkan lewat kartu kredit, sementara cash tip tidak tercatat
- Tolls_amount
    - Jumlah biaya tol yang dibayar selama perjalanan
- Total_amount
    - Total biaya yang perlu dibayar oleh penumpang setelah perjalanan. Cash tip dari penumpang tidak tercatat di data ini
- Trip_type
    - ID yang menandakan jenis perjalanan

In [ ]:
# Load dataset ke dalam variabel df
df = pd.read_csv("NYC TLC Trip Record.csv")
display(df.head(), df.tail())

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,NaN,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,NaN,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,NaN,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,NaN,1.0,8.00,1.0,1.0,0.00


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
68206,2,2023-01-31 22:29:00,2023-01-31 22:42:00,NaN,NaN,49,62,NaN,4070.82,15.70,0.0,0.0,0.00,0.0,NaN,1.0,16.70,NaN,NaN,NaN
68207,2,2023-01-31 22:40:00,2023-01-31 22:48:00,NaN,NaN,10,205,NaN,2.14,4.41,0.0,0.0,0.00,0.0,NaN,1.0,5.41,NaN,NaN,NaN
68208,2,2023-01-31 23:46:00,2023-02-01 00:02:00,NaN,NaN,66,37,NaN,3.44,16.53,0.0,0.0,3.51,0.0,NaN,1.0,21.04,NaN,NaN,NaN
68209,2,2023-01-31 23:01:00,2023-01-31 23:19:00,NaN,NaN,225,189,NaN,3.03,14.98,0.0,0.0,3.20,0.0,NaN,1.0,19.18,NaN,NaN,NaN
68210,2,2023-01-31 23:51:00,2023-02-01 00:07:00,NaN,NaN,256,140,NaN,5.82,23.65,0.0,0.0,2.00,0.0,NaN,1.0,29.40,NaN,NaN,NaN


## **2.1. Data Understanding and Cleaning**


Sebelum data bisa dianalisis, data yang dimiliki perlu diamati dan dikenali terlebih dahulu melalui proses *data understanding*. Hal ini diperlukan untuk menemukan anomali-anomali pada data yang nantinya dapat mengganggu proses analisis data, serta dapat menyebabkan hasil analisis yang dilakukan menjadi tidak dapat menggambarkan kondisi asli dengan baik. 

Berikut merupakan karateristik dari dataset "NYC TLC Trip Record.csv"

In [ ]:
# Menampilkan baris dan kolom dari dataset
print("Jumlah baris dan kolom dari dataset:", df.shape)

Jumlah baris dan kolom dari dataset: (68211, 20)


In [ ]:
# Menampilkan informasi umum pada dataset ini
print("Informasi umum pada dataset ini:")
df.info()

Informasi umum pada dataset ini:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68211 entries, 0 to 68210
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               68211 non-null  int64  
 1   lpep_pickup_datetime   68211 non-null  object 
 2   lpep_dropoff_datetime  68211 non-null  object 
 3   store_and_fwd_flag     63887 non-null  object 
 4   RatecodeID             63887 non-null  float64
 5   PULocationID           68211 non-null  int64  
 6   DOLocationID           68211 non-null  int64  
 7   passenger_count        63887 non-null  float64
 8   trip_distance          68211 non-null  float64
 9   fare_amount            68211 non-null  float64
 10  extra                  68211 non-null  float64
 11  mta_tax                68211 non-null  float64
 12  tip_amount             68211 non-null  float64
 13  tolls_amount           68211 non-null  float64
 14  ehail_fee            

In [ ]:
# Menampilkan deskripsi statistik dari dataset
display(df.describe(), df.describe(include="object"))

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,68211.000000,63887.000000,68211.000000,68211.000000,63887.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,0.0,68211.000000,68211.000000,63887.000000,63877.000000,63887.000000
mean,1.863028,1.117160,98.549735,138.429901,1.315870,8.114852,16.603545,0.825431,0.588340,2.139012,0.169627,NaN,0.951625,21.789377,1.373613,1.021933,0.726748
std,0.343820,1.372913,61.244314,76.761311,0.979054,585.105955,13.470121,1.269904,0.385819,3.052710,1.099789,NaN,0.196875,15.457115,0.508039,0.146465,1.212836
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-70.000000,-2.500000,-0.500000,-10.500000,0.000000,NaN,-1.000000,-71.500000,1.000000,1.000000,-2.750000
25%,2.000000,1.000000,74.000000,74.000000,1.000000,1.110000,9.300000,0.000000,0.500000,0.000000,0.000000,NaN,1.000000,12.900000,1.000000,1.000000,0.000000
50%,2.000000,1.000000,75.000000,138.000000,1.000000,1.850000,13.500000,0.000000,0.500000,1.600000,0.000000,NaN,1.000000,17.900000,1.000000,1.000000,0.000000
75%,2.000000,1.000000,129.000000,219.000000,1.000000,3.210000,19.800000,1.000000,0.500000,3.330000,0.000000,NaN,1.000000,26.150000,2.000000,1.000000,2.750000
max,2.000000,99.000000,265.000000,265.000000,9.000000,120098.840000,490.000000,12.500000,2.750000,222.220000,36.050000,NaN,1.000000,491.000000,5.000000,2.000000,2.750000


,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag
count,68211,68211,63887
unique,66575,66519,2
top,2023-01-07 23:58:06,2023-01-06 00:00:00,N
freq,4,6,63579


In [ ]:
# Menampilkan tiap data unik dalam tiap kolom pada dataset
pd.set_option("display.max_colwidth", 500)
listItem = []
for col in df.columns :
    listItem.append( [col, df[col].nunique(), df[col].unique()])

tabel1Desc = pd.DataFrame(columns=["Column Name", "Number of Unique", "Unique Sample"],
                     data=listItem)
tabel1Desc

,Column Name,Number of Unique,Unique Sample
0,VendorID,2,"[2, 1]"
1,lpep_pickup_datetime,66575,"[2023-01-01 00:26:10, 2023-01-01 00:51:03, 2023-01-01 00:35:12, 2023-01-01 00:13:14, 2023-01-01 00:33:04, 2023-01-01 00:53:31, 2023-01-01 00:09:14, 2023-01-01 00:11:58, 2023-01-01 00:41:29, 2023-01-01 00:50:32, 2023-01-01 00:16:12, 2023-01-01 00:08:43, 2023-01-01 00:26:32, 2023-01-01 00:18:35, 2023-01-01 00:39:32, 2023-01-01 00:49:34, 2023-01-01 00:10:45, 2023-01-01 00:35:11, 2023-01-01 00:31:06, 2023-01-01 00:14:37, 2023-01-01 00:26:21, 2023-01-01 00:41:56, 2023-01-01 00:32:02, 2023-01-01 0..."
2,lpep_dropoff_datetime,66519,"[2023-01-01 00:37:11, 2023-01-01 00:57:49, 2023-01-01 00:41:32, 2023-01-01 00:19:03, 2023-01-01 00:39:02, 2023-01-01 01:11:04, 2023-01-01 00:26:39, 2023-01-01 00:24:55, 2023-01-01 00:46:26, 2023-01-01 01:13:42, 2023-01-01 00:41:43, 2023-01-01 00:17:08, 2023-01-01 00:45:31, 2023-01-01 00:30:09, 2023-01-01 01:18:06, 2023-01-01 01:08:23, 2023-01-01 00:19:37, 2023-01-01 01:17:35, 2023-01-01 00:42:23, 2023-01-01 00:21:50, 2023-01-01 00:30:26, 2023-01-01 00:46:06, 2023-01-01 00:41:25, 2023-01-01 0..."
3,store_and_fwd_flag,2,"[N, Y, nan]"
4,RatecodeID,6,"[1.0, 5.0, 4.0, 3.0, 2.0, 99.0, nan]"
5,PULocationID,226,"[166, 24, 223, 41, 181, 255, 75, 66, 195, 83, 97, 7, 42, 95, 260, 134, 130, 258, 129, 116, 92, 65, 74, 17, 152, 210, 112, 69, 243, 82, 179, 168, 80, 25, 145, 264, 33, 226, 52, 169, 36, 51, 157, 43, 256, 173, 70, 251, 40, 101, 61, 241, 146, 147, 228, 171, 138, 56, 244, 263, 196, 37, 236, 213, 28, 117, 14, 188, 177, 29, 262, 250, 72, 216, 22, 20, 139, 55, 257, 193, 127, 219, 3, 119, 265, 167, 150, 78, 135, 247, 93, 159, 48, 31, 91, 53, 160, 94, 47, 254, ...]"
6,DOLocationID,249,"[143, 43, 179, 238, 74, 262, 45, 75, 166, 140, 234, 148, 255, 186, 210, 129, 68, 260, 42, 24, 244, 146, 135, 173, 216, 142, 265, 164, 7, 83, 95, 130, 223, 80, 48, 263, 196, 250, 226, 239, 189, 41, 168, 162, 141, 33, 145, 151, 116, 195, 249, 235, 160, 98, 79, 229, 181, 193, 237, 233, 264, 102, 10, 37, 258, 82, 170, 61, 112, 231, 97, 17, 169, 96, 219, 243, 236, 56, 36, 114, 157, 161, 54, 90, 40, 188, 70, 63, 251, 101, 92, 153, 53, 57, 256, 65, 257, 138, 85, 52, ...]"
7,passenger_count,10,"[1.0, 2.0, 4.0, 3.0, 6.0, 5.0, 0.0, 7.0, 9.0, 8.0, nan]"
8,trip_distance,1870,"[2.58, 1.81, 0.0, 1.3, 1.1, 2.78, 3.8, 1.88, 1.11, 4.22, 4.8, 1.99, 7.08, 3.23, 5.91, 11.3, 8.28, 2.02, 0.96, 0.53, 1.51, 0.88, 2.48, 3.53, 2.25, 4.29, 2.53, 1.7, 10.44, 6.78, 1.39, 1.31, 6.7, 3.52, 1.74, 2.31, 1.37, 10.35, 1.05, 6.19, 2.8, 1.13, 3.19, 1.26, 2.87, 8.55, 4.46, 1.18, 2.5, 9.01, 0.63, 2.08, 1.47, 7.83, 2.15, 12.73, 1.62, 7.3, 2.73, 4.9, 2.17, 0.6, 0.8, 2.39, 1.23, 10.45, 2.23, 2.21, 5.2, 5.25, 6.32, 3.82, 3.58, 0.7, 2.91, 5.13, 12.4, 1.45, 7.52, 2.45, 0.54, 0.47, 1.16, 15.39, 5..."
9,fare_amount,2553,"[14.9, 10.7, 7.2, 6.5, 6.0, 17.7, 19.1, 14.2, 24.7, 26.8, 11.4, 30.3, 35.9, 32.5, 15.0, 44.3, 12.8, 8.6, 5.8, 16.3, 19.8, 18.4, 13.5, 8.0, 47.1, 60.0, 7.9, 10.0, 3.0, 5.5, 17.0, 75.0, 15.6, 47.8, 21.2, 37.3, 52.0, 4.4, 51.3, 25.5, 5.1, 9.3, 29.6, 48.5, 28.2, 21.9, 5.0, 4.0, 25.4, 36.6, 66.0, 20.0, 50.0, 190.0, 20.5, 46.4, 12.1, 31.0, 33.8, 26.1, 22.6, 15.5, 24.0, 25.0, 10.5, 19.5, 100.0, 40.0, 31.7, 40.8, 80.0, 110.0, 12.0, 14.5, 43.6, 23.0, 90.0, 49.2, 16.0, 28.9, 30.0, 25.8, 35.0, 35.2, 7...."


Dapat dilihat bahwa:
- Dataset "NYC TLC Trip Record.csv" terdiri atas 68211 baris dan 20 kolom. 
- Dataset masih mengandung beberapa nilai kosong di beberapa kolom, yang nantinya dapat mengganggu proses analisis data.
- Terdapat kolom yang tidak ada di dalam Data Dictionary yang diberikan oleh pihak New York City TLC, yaitu kolom `ehail_fee`. Tidak hanya itu, kolom ini hanya mengandung nilai kosong saja. 
- Data pada kolom `lpep_pickup_datetime` dan `lpep_dropoff_datetime` memiliki tipe data "object". Karena data tersebut menggambarkan tanggal dan waktu mulai dan selesainya layanan antar-jemput, data tersebut seharusnya bertipe "datetime".
- Pada kolom `RatecodeID` terdapat data bernilai 99.0. Hal ini tidak seharusnya terjadi, karena pada Data Dictionary yang diberikan oleh pihak New York City TLC, kolom tersebut seharusnya hanya memiliki nilai angka dari 1 sampai 6.
- Pada kolom `extra`, `mta_tax`, `improvement_surcharge` serta `congestion_surcharge` mengandung beberapa nilai negatif. Karena data-data tersebut menggambarkan biaya-biaya tambahan yang perlu dibayar oleh penumpang, nilai dari data tersebut seharusnya konsisten positif atau negatif.

Dari informasi di atas, dapat disimpulkan bahwa sebelum analisis data dapat dilakukan, data perlu diolah terlebih dahulu melalui proses *data cleaning*, yang dilakukan dengan cara:
- Menghilangkan nilai-nilai kosong (*missing values*) pada data.
- Menghilangkan kolom `ehail_fee`, karena kolom tersebut tidak memiliki efek pada data-data lain dari dataset serta hanya akan mengganggu proses analisis data karena hanya berisi data kosong.
- Mengubah nilai pada kolom `lpep_pickup_datetime` dan `lpep_dropoff_datetime` menjadi tipe datetime.
- Menghilangkan atau mengubah data-data pada kolom `RatecodeID` yang bernilai 99.0
- Menyelaraskan nilai-nilai pada kolom`extra`, `mta_tax`, `improvement_surcharge` dan `congestion_surcharge`.

### Missing Value

In [ ]:
# Membuat variabel baru memuat dataset yang ingin dibersihkan
df_clean = df.copy()

In [ ]:
# Mencari nilai kosong di dalam tiap kolom pada dataset
df_clean.isna().sum()

VendorID                     0
lpep_pickup_datetime         0
lpep_dropoff_datetime        0
store_and_fwd_flag        4324
RatecodeID                4324
PULocationID                 0
DOLocationID                 0
passenger_count           4324
trip_distance                0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
ehail_fee                68211
improvement_surcharge        0
total_amount                 0
payment_type              4324
trip_type                 4334
congestion_surcharge      4324
dtype: int64

In [ ]:
# Melihat persentase nilai kosong di dalam tiap kolom pada dataset
df_clean.isna().sum()/df_clean.shape[0]*100

VendorID                   0.000000
lpep_pickup_datetime       0.000000
lpep_dropoff_datetime      0.000000
store_and_fwd_flag         6.339154
RatecodeID                 6.339154
PULocationID               0.000000
DOLocationID               0.000000
passenger_count            6.339154
trip_distance              0.000000
fare_amount                0.000000
extra                      0.000000
mta_tax                    0.000000
tip_amount                 0.000000
tolls_amount               0.000000
ehail_fee                100.000000
improvement_surcharge      0.000000
total_amount               0.000000
payment_type               6.339154
trip_type                  6.353814
congestion_surcharge       6.339154
dtype: float64

Jika dilihat dari data persentasi nilai kosong di dalam tiap kolom pada dataset, dapat terlihat bahwa: 
- Nilai kosong terdapat pada kolom `store_and_fwd_flag`, `RatecodeID`, `passenger_count`, `ehail_fee`, `payment_type`, `trip_type` dan `congestion_surcharge`.
- Nilai kosong pada kolom `ehail_fee` berjumlah sebanyak **68211 buah** dan menyusun sebesar **100%** dari data pada kolom tersebut. 
    - Seperti yang telah ditemukan sebelumnya, hal ini menandakan bahwa kolom ini sepenuhnya kosong. 
- Nilai kosong pada kolom `store_and_fwd_flag`, `RatecodeID`, `passenger_count`, `payment_type` dan `congestion_surcharge` berjumlah sama, yaitu sebanyak **4324 buah**, dimana nilai-nilai kosong ini menyusun sebesar **6.34%** dari data pada tiap kolom tersebut. 
    - Karena jumlah nilai kosong pada kolom-kolom tersebut persis sama, nilai-nilai kosong ini mungkin berkaitan.
- Nilai kosong pada kolom `trip_type` memiliki jumlah yang berbeda sedikit dengan pada kolom-kolom di atas, yaitu sebanyak **4334 buah** (10 lebih banyak dibandingkan dengan nilai kosong pada kolom-kolom di atas) dimana nilai-nilai kosong ini menyusun sebesar **6.35%** dari data pada tiap kolom tersebut. 
    - Perbedaan nilai ini menandakan bahwa paling tidak beberapa dari nilai kosong pada kolom `trip_type` mungkin tidak berkaitan dengan nilai-nilai kosong dari kolom lain, sehingga perlu ditinjau secara terpisah.
- Nilai kosong pada tiap kolom pada dataset (kecuali pada kolom `ehail_fee`) berjumlah sangat kecil (<7%), sehingga dapat dihapus tanpa mempengaruhi analisis data secara terlalu banyak.

Untuk memulai proses pengolahan nilai kosong, mari kita hapus kolom `ehail_fee` dari dataset.

In [ ]:
# Menghapus kolom `ehail_fee`
df_clean.drop(columns='ehail_fee', inplace=True)

Setelah itu, kita bisa meninjau nilai-nilai kosong pada kolom-kolom `store_and_fwd_flag`, `RatecodeID`, `passenger_count`, `payment_type` dan `congestion_surcharge`, untuk melihat apakah ada hubungan di antara semua nilai kosong tersebut. Peninjauan ini dapat kita mulai dengan mengamati tampilan 10 data paling awal dari kolom-kolom tersebut yang memiliki nilai kosong.  

In [1408]:
df[df['payment_type'].isnull() == True].head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
63887,2,2023-01-01 00:28:00,2023-01-01 00:54:00,NaN,NaN,7,17,NaN,9.33,36.03,0.0,0.0,7.41,0.0,NaN,1.0,44.44,NaN,NaN,NaN
63888,2,2023-01-01 00:39:00,2023-01-01 00:52:00,NaN,NaN,37,37,NaN,2.67,14.55,0.0,0.0,3.11,0.0,NaN,1.0,18.66,NaN,NaN,NaN
63889,2,2023-01-01 00:58:00,2023-01-01 01:21:00,NaN,NaN,80,7,NaN,7.39,32.13,0.0,0.0,6.63,0.0,NaN,1.0,39.76,NaN,NaN,NaN
63890,2,2023-01-01 00:35:00,2023-01-01 00:55:00,NaN,NaN,169,42,NaN,4.04,17.85,0.0,0.0,0.00,0.0,NaN,1.0,18.85,NaN,NaN,NaN
63891,2,2023-01-01 00:45:00,2023-01-01 00:55:00,NaN,NaN,42,262,NaN,3.86,19.67,0.0,0.0,4.68,0.0,NaN,1.0,28.10,NaN,NaN,NaN
63892,2,2023-01-01 00:30:00,2023-01-01 01:01:00,NaN,NaN,7,61,NaN,15.43,55.53,0.0,0.0,16.96,0.0,NaN,1.0,73.49,NaN,NaN,NaN
63893,2,2023-01-01 00:32:00,2023-01-01 01:00:00,NaN,NaN,24,113,NaN,6.63,35.40,0.0,0.0,7.83,0.0,NaN,1.0,46.98,NaN,NaN,NaN
63894,2,2023-01-01 00:28:00,2023-01-01 00:46:00,NaN,NaN,74,68,NaN,7.28,33.29,0.0,0.0,2.00,0.0,NaN,1.0,39.04,NaN,NaN,NaN
63895,2,2023-01-01 00:33:00,2023-01-01 00:43:00,NaN,NaN,52,181,NaN,1.86,12.90,0.0,0.0,2.78,0.0,NaN,1.0,16.68,NaN,NaN,NaN
63896,2,2023-01-01 00:59:00,2023-01-01 01:12:00,NaN,NaN,65,211,NaN,2.60,15.43,0.0,0.0,3.84,0.0,NaN,1.0,23.02,NaN,NaN,NaN


Pada tampilan tersebut, terlihat bahwa apabila nilai pada salah satu dari kolom `store_and_fwd_flag`, `RatecodeID`, `passenger_count`, `payment_type` dan `congestion_surcharge` kosong, nilai pada ke-empat kolom lainnya pun juga akan kosong. Untuk memastikan apakah hubungan ini menggambarkan tiap nilai kosong pada ke-lima kolom tersebut, mari kita lihat jumlah nilai kosong pada baris-baris dataset yang bernilai kosong pada kolom `payment_type`.

In [ ]:
# Jumlah nilai kosong pada tiap kolom yang nilai "payment_type"
df[df['payment_type'].isnull() == True].isnull().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
store_and_fwd_flag       4324
RatecodeID               4324
PULocationID                0
DOLocationID                0
passenger_count          4324
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
ehail_fee                4324
improvement_surcharge       0
total_amount                0
payment_type             4324
trip_type                4324
congestion_surcharge     4324
dtype: int64

Pada tampilan tersebut, dapat terlihat bahwa jumlah nilai kosong pada kolom `store_and_fwd_flag`, `RatecodeID`, `passenger_count` dan `congestion_surcharge` ketika nilai pada kolom `payment_type` kosong adalah sebesar 4324 buah, yang merupakan jumlah nilai kosong yang sama pada kolom-kolom tersebut ketika jumlah nilai kosong dalam dataset ditinjau secara keseluruhan. Tidak hanya itu, kolom `trip_type` juga menunjukkan jumlah nilai kosong sebesar 4324 buah. Hal ini menunjukkan bahwa setiap dari 4324 nilai kosong pada kolom-kolom tersebut mungkin **berhubungan**.

In [ ]:
# Menghapus nilai kosong dari dataset dimana "payment_type" bernilai kosong
df_clean.drop(index=(df_clean[df_clean['payment_type'].isnull() == True].index), inplace=True)
df_clean.isnull().sum()

VendorID                  0
lpep_pickup_datetime      0
lpep_dropoff_datetime     0
store_and_fwd_flag        0
RatecodeID                0
PULocationID              0
DOLocationID              0
passenger_count           0
trip_distance             0
fare_amount               0
extra                     0
mta_tax                   0
tip_amount                0
tolls_amount              0
improvement_surcharge     0
total_amount              0
payment_type              0
trip_type                10
congestion_surcharge      0
dtype: int64

In [1350]:
list_item = []
for col in df_clean.columns:
    list_item.append([col, df_clean[col].nunique(), df_clean[col].unique()])

df_clean_unique = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)
df_clean_unique

,Column Name,Number of Unique,Unique Sample
0,VendorID,2,"[2, 1]"
1,lpep_pickup_datetime,62675,"[2023-01-01 00:26:10, 2023-01-01 00:51:03, 2023-01-01 00:35:12, 2023-01-01 00:13:14, 2023-01-01 00:33:04, 2023-01-01 00:53:31, 2023-01-01 00:09:14, 2023-01-01 00:11:58, 2023-01-01 00:41:29, 2023-01-01 00:50:32, 2023-01-01 00:16:12, 2023-01-01 00:08:43, 2023-01-01 00:26:32, 2023-01-01 00:18:35, 2023-01-01 00:39:32, 2023-01-01 00:49:34, 2023-01-01 00:10:45, 2023-01-01 00:35:11, 2023-01-01 00:31:06, 2023-01-01 00:14:37, 2023-01-01 00:26:21, 2023-01-01 00:41:56, 2023-01-01 00:32:02, 2023-01-01 0..."
2,lpep_dropoff_datetime,62612,"[2023-01-01 00:37:11, 2023-01-01 00:57:49, 2023-01-01 00:41:32, 2023-01-01 00:19:03, 2023-01-01 00:39:02, 2023-01-01 01:11:04, 2023-01-01 00:26:39, 2023-01-01 00:24:55, 2023-01-01 00:46:26, 2023-01-01 01:13:42, 2023-01-01 00:41:43, 2023-01-01 00:17:08, 2023-01-01 00:45:31, 2023-01-01 00:30:09, 2023-01-01 01:18:06, 2023-01-01 01:08:23, 2023-01-01 00:19:37, 2023-01-01 01:17:35, 2023-01-01 00:42:23, 2023-01-01 00:21:50, 2023-01-01 00:30:26, 2023-01-01 00:46:06, 2023-01-01 00:41:25, 2023-01-01 0..."
3,store_and_fwd_flag,2,"[N, Y]"
4,RatecodeID,6,"[1.0, 5.0, 4.0, 3.0, 2.0, 99.0]"
5,PULocationID,222,"[166, 24, 223, 41, 181, 255, 75, 66, 195, 83, 97, 7, 42, 95, 260, 134, 130, 258, 129, 116, 92, 65, 74, 17, 152, 210, 112, 69, 243, 82, 179, 168, 80, 25, 145, 264, 33, 226, 52, 169, 36, 51, 157, 43, 256, 173, 70, 251, 40, 101, 61, 241, 146, 147, 228, 171, 138, 56, 244, 263, 196, 37, 236, 213, 28, 117, 14, 188, 177, 29, 262, 250, 72, 216, 22, 20, 139, 55, 257, 193, 127, 219, 3, 119, 265, 167, 150, 78, 135, 247, 93, 159, 48, 31, 91, 53, 160, 94, 47, 254, ...]"
6,DOLocationID,248,"[143, 43, 179, 238, 74, 262, 45, 75, 166, 140, 234, 148, 255, 186, 210, 129, 68, 260, 42, 24, 244, 146, 135, 173, 216, 142, 265, 164, 7, 83, 95, 130, 223, 80, 48, 263, 196, 250, 226, 239, 189, 41, 168, 162, 141, 33, 145, 151, 116, 195, 249, 235, 160, 98, 79, 229, 181, 193, 237, 233, 264, 102, 10, 37, 258, 82, 170, 61, 112, 231, 97, 17, 169, 96, 219, 243, 236, 56, 36, 114, 157, 161, 54, 90, 40, 188, 70, 63, 251, 101, 92, 153, 53, 57, 256, 65, 257, 138, 85, 52, ...]"
7,passenger_count,10,"[1.0, 2.0, 4.0, 3.0, 6.0, 5.0, 0.0, 7.0, 9.0, 8.0]"
8,trip_distance,1728,"[2.58, 1.81, 0.0, 1.3, 1.1, 2.78, 3.8, 1.88, 1.11, 4.22, 4.8, 1.99, 7.08, 3.23, 5.91, 11.3, 8.28, 2.02, 0.96, 0.53, 1.51, 0.88, 2.48, 3.53, 2.25, 4.29, 2.53, 1.7, 10.44, 6.78, 1.39, 1.31, 6.7, 3.52, 1.74, 2.31, 1.37, 10.35, 1.05, 6.19, 2.8, 1.13, 3.19, 1.26, 2.87, 8.55, 4.46, 1.18, 2.5, 9.01, 0.63, 2.08, 1.47, 7.83, 2.15, 12.73, 1.62, 7.3, 2.73, 4.9, 2.17, 0.6, 0.8, 2.39, 1.23, 10.45, 2.23, 2.21, 5.2, 5.25, 6.32, 3.82, 3.58, 0.7, 2.91, 5.13, 12.4, 1.45, 7.52, 2.45, 0.54, 0.47, 1.16, 15.39, 5..."
9,fare_amount,463,"[14.9, 10.7, 7.2, 6.5, 6.0, 17.7, 19.1, 14.2, 24.7, 26.8, 11.4, 30.3, 35.9, 32.5, 15.0, 44.3, 12.8, 8.6, 5.8, 16.3, 19.8, 18.4, 13.5, 8.0, 47.1, 60.0, 7.9, 10.0, 3.0, 5.5, 17.0, 75.0, 15.6, 47.8, 21.2, 37.3, 52.0, 4.4, 51.3, 25.5, 5.1, 9.3, 29.6, 48.5, 28.2, 21.9, 5.0, 4.0, 25.4, 36.6, 66.0, 20.0, 50.0, 190.0, 20.5, 46.4, 12.1, 31.0, 33.8, 26.1, 22.6, 15.5, 24.0, 25.0, 10.5, 19.5, 100.0, 40.0, 31.7, 40.8, 80.0, 110.0, 12.0, 14.5, 43.6, 23.0, 90.0, 49.2, 16.0, 28.9, 30.0, 25.8, 35.0, 35.2, 7...."


In [1351]:
df_clean[df_clean['RatecodeID']==99.0]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
1084,1,2023-01-01 19:44:13,2023-01-01 19:58:30,N,99.0,150,55,1.0,3.3,18.2,0.0,1.5,0.0,0.0,1.0,19.7,1.0,NaN,0.0
2174,1,2023-01-02 16:39:05,2023-01-02 17:31:42,N,99.0,73,82,1.0,3.7,23.5,0.0,1.5,0.0,0.0,1.0,25.0,1.0,NaN,0.0
9055,1,2023-01-05 20:33:45,2023-01-05 22:46:41,N,99.0,14,123,1.0,6.2,28.2,0.0,1.5,0.0,0.0,1.0,29.7,1.0,NaN,0.0
14125,1,2023-01-08 11:36:41,2023-01-08 12:14:59,N,99.0,61,77,1.0,2.7,22.5,0.0,1.5,0.0,0.0,1.0,24.0,1.0,NaN,0.0
29143,1,2023-01-15 13:51:42,2023-01-15 14:37:38,N,99.0,108,181,1.0,6.5,24.2,0.0,1.5,0.0,0.0,1.0,25.7,1.0,NaN,0.0
29711,1,2023-01-15 17:52:52,2023-01-15 22:11:55,N,99.0,150,123,1.0,1.6,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
35975,1,2023-01-19 01:07:59,2023-01-19 01:17:19,N,99.0,90,68,1.0,0.6,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
37691,1,2023-01-19 17:16:59,2023-01-19 22:29:53,N,99.0,265,123,1.0,14.6,40.2,0.0,1.5,0.0,0.0,1.0,41.7,1.0,NaN,0.0
42432,1,2023-01-21 20:29:58,2023-01-22 02:59:25,N,99.0,150,123,1.0,1.4,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
45204,1,2023-01-23 13:45:11,2023-01-23 23:36:42,N,99.0,244,248,1.0,28.3,50.2,0.0,1.5,0.0,0.0,1.0,51.7,1.0,NaN,0.0


In [1352]:
df_clean[df_clean['trip_type'].isnull()==True]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
1084,1,2023-01-01 19:44:13,2023-01-01 19:58:30,N,99.0,150,55,1.0,3.3,18.2,0.0,1.5,0.0,0.0,1.0,19.7,1.0,NaN,0.0
2174,1,2023-01-02 16:39:05,2023-01-02 17:31:42,N,99.0,73,82,1.0,3.7,23.5,0.0,1.5,0.0,0.0,1.0,25.0,1.0,NaN,0.0
9055,1,2023-01-05 20:33:45,2023-01-05 22:46:41,N,99.0,14,123,1.0,6.2,28.2,0.0,1.5,0.0,0.0,1.0,29.7,1.0,NaN,0.0
14125,1,2023-01-08 11:36:41,2023-01-08 12:14:59,N,99.0,61,77,1.0,2.7,22.5,0.0,1.5,0.0,0.0,1.0,24.0,1.0,NaN,0.0
29143,1,2023-01-15 13:51:42,2023-01-15 14:37:38,N,99.0,108,181,1.0,6.5,24.2,0.0,1.5,0.0,0.0,1.0,25.7,1.0,NaN,0.0
29711,1,2023-01-15 17:52:52,2023-01-15 22:11:55,N,99.0,150,123,1.0,1.6,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
35975,1,2023-01-19 01:07:59,2023-01-19 01:17:19,N,99.0,90,68,1.0,0.6,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
37691,1,2023-01-19 17:16:59,2023-01-19 22:29:53,N,99.0,265,123,1.0,14.6,40.2,0.0,1.5,0.0,0.0,1.0,41.7,1.0,NaN,0.0
42432,1,2023-01-21 20:29:58,2023-01-22 02:59:25,N,99.0,150,123,1.0,1.4,15.2,0.0,1.5,0.0,0.0,1.0,16.7,1.0,NaN,0.0
45204,1,2023-01-23 13:45:11,2023-01-23 23:36:42,N,99.0,244,248,1.0,28.3,50.2,0.0,1.5,0.0,0.0,1.0,51.7,1.0,NaN,0.0


In [1353]:
df_clean.drop(index=(df_clean[df_clean['trip_type'].isnull() == True].index), inplace=True)
df_clean.isnull().sum()

VendorID                 0
lpep_pickup_datetime     0
lpep_dropoff_datetime    0
store_and_fwd_flag       0
RatecodeID               0
PULocationID             0
DOLocationID             0
passenger_count          0
trip_distance            0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
payment_type             0
trip_type                0
congestion_surcharge     0
dtype: int64

### Duplicated Values

In [1354]:
df_clean.duplicated().sum()

np.int64(0)

### Odd Values

#### Wrong Datetime

In [1355]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63877 entries, 0 to 63886
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               63877 non-null  int64  
 1   lpep_pickup_datetime   63877 non-null  object 
 2   lpep_dropoff_datetime  63877 non-null  object 
 3   store_and_fwd_flag     63877 non-null  object 
 4   RatecodeID             63877 non-null  float64
 5   PULocationID           63877 non-null  int64  
 6   DOLocationID           63877 non-null  int64  
 7   passenger_count        63877 non-null  float64
 8   trip_distance          63877 non-null  float64
 9   fare_amount            63877 non-null  float64
 10  extra                  63877 non-null  float64
 11  mta_tax                63877 non-null  float64
 12  tip_amount             63877 non-null  float64
 13  tolls_amount           63877 non-null  float64
 14  improvement_surcharge  63877 non-null  float64
 15  total_a

In [1356]:
df_clean['lpep_pickup_datetime'] = pd.to_datetime(df_clean['lpep_pickup_datetime'])
df_clean['lpep_dropoff_datetime'] = pd.to_datetime(df_clean['lpep_dropoff_datetime'])

In [1357]:
df_clean[['lpep_pickup_datetime', 'lpep_dropoff_datetime']].sort_values(by='lpep_pickup_datetime', ascending=True)

,lpep_pickup_datetime,lpep_dropoff_datetime
17874,2009-01-01 20:21:27,2009-01-02 11:07:31
298,2022-12-09 14:10:49,2022-12-09 14:22:26
318,2022-12-09 14:55:42,2022-12-09 15:33:06
24,2023-01-01 00:01:31,2023-01-01 00:16:02
44,2023-01-01 00:04:25,2023-01-01 00:17:50
...,...,...
63844,2023-01-31 23:45:10,2023-01-31 23:54:53
63876,2023-01-31 23:53:41,2023-02-01 00:31:14
63858,2023-01-31 23:56:16,2023-02-01 00:03:16
63872,2023-01-31 23:58:23,2023-02-01 00:07:16


In [1358]:
df_clean['lpep_pickup_datetime'].dt.year.unique()

array([2023, 2022, 2009], dtype=int32)

In [1359]:
print("Bulan yang tercatat dalam data: ")
print("Tahun 2022: ", df_clean['lpep_pickup_datetime'][df_clean['lpep_dropoff_datetime'].dt.year == 2022].dt.month.unique())
print("Tahun 2023: ", df_clean['lpep_pickup_datetime'][df_clean['lpep_dropoff_datetime'].dt.year == 2023].dt.month.unique())


Bulan yang tercatat dalam data: 
Tahun 2022:  [12]
Tahun 2023:  [1 2]


In [1360]:
print("Hari yang tercatat dalam data: ")
print("Tahun 2022: ", df_clean['lpep_pickup_datetime'][df_clean['lpep_dropoff_datetime'].dt.year == 2022].dt.day.unique())
print("Tahun 2023: ", df_clean['lpep_pickup_datetime'][df_clean['lpep_dropoff_datetime'].dt.year == 2023].dt.day.unique())


Hari yang tercatat dalam data: 
Tahun 2022:  [9]
Tahun 2023:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]


In [1361]:
df_clean.drop(index=(df_clean[df_clean['lpep_dropoff_datetime'].dt.year < 2023].index), inplace=True)


In [1362]:
df_clean['lpep_pickup_datetime'].dt.year.unique()

array([2023], dtype=int32)

In [1363]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63874 entries, 0 to 63886
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               63874 non-null  int64         
 1   lpep_pickup_datetime   63874 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  63874 non-null  datetime64[ns]
 3   store_and_fwd_flag     63874 non-null  object        
 4   RatecodeID             63874 non-null  float64       
 5   PULocationID           63874 non-null  int64         
 6   DOLocationID           63874 non-null  int64         
 7   passenger_count        63874 non-null  float64       
 8   trip_distance          63874 non-null  float64       
 9   fare_amount            63874 non-null  float64       
 10  extra                  63874 non-null  float64       
 11  mta_tax                63874 non-null  float64       
 12  tip_amount             63874 non-null  float64       
 13  tolls_

#### Unknown Pickup and Dropoff Location

In [1364]:
df_clean.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,1.0,8.00,1.0,1.0,0.00


In [1365]:
pickup_location = np.sort(df_clean['PULocationID'].unique())
pickup_location

array([  1,   3,   7,   9,  10,  11,  14,  16,  17,  18,  19,  20,  21,
        22,  23,  24,  25,  26,  27,  28,  29,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  97,  98, 100, 101, 102, 106, 108, 112,
       113, 114, 116, 117, 119, 120, 121, 122, 123, 124, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 143,
       145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 157, 159, 160,
       161, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 177, 178,
       179, 180, 181, 182, 183, 185, 186, 188, 189, 190, 191, 192, 193,
       194, 195, 196, 197, 198, 200, 201, 202, 203, 205, 206, 207, 208,
       209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220, 22

In [1366]:
dropout_location = np.sort(df_clean['DOLocationID'].unique())
dropout_location

array([  1,   3,   4,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98, 100, 101, 102, 106, 107, 108, 111, 112, 113, 114, 115, 116,
       117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 19

In [1367]:
df_clean[df_clean['PULocationID']==264].count()

VendorID                 135
lpep_pickup_datetime     135
lpep_dropoff_datetime    135
store_and_fwd_flag       135
RatecodeID               135
PULocationID             135
DOLocationID             135
passenger_count          135
trip_distance            135
fare_amount              135
extra                    135
mta_tax                  135
tip_amount               135
tolls_amount             135
improvement_surcharge    135
total_amount             135
payment_type             135
trip_type                135
congestion_surcharge     135
dtype: int64

In [1368]:
df_clean[df_clean['DOLocationID']==264].count()

VendorID                 424
lpep_pickup_datetime     424
lpep_dropoff_datetime    424
store_and_fwd_flag       424
RatecodeID               424
PULocationID             424
DOLocationID             424
passenger_count          424
trip_distance            424
fare_amount              424
extra                    424
mta_tax                  424
tip_amount               424
tolls_amount             424
improvement_surcharge    424
total_amount             424
payment_type             424
trip_type                424
congestion_surcharge     424
dtype: int64

In [1369]:
df_clean[(df_clean['DOLocationID']==264) & (df_clean['PULocationID']==264)].count()

VendorID                 131
lpep_pickup_datetime     131
lpep_dropoff_datetime    131
store_and_fwd_flag       131
RatecodeID               131
PULocationID             131
DOLocationID             131
passenger_count          131
trip_distance            131
fare_amount              131
extra                    131
mta_tax                  131
tip_amount               131
tolls_amount             131
improvement_surcharge    131
total_amount             131
payment_type             131
trip_type                131
congestion_surcharge     131
dtype: int64

In [1370]:
df_clean[(df_clean['DOLocationID']==264) | (df_clean['PULocationID']==264)].count()

VendorID                 428
lpep_pickup_datetime     428
lpep_dropoff_datetime    428
store_and_fwd_flag       428
RatecodeID               428
PULocationID             428
DOLocationID             428
passenger_count          428
trip_distance            428
fare_amount              428
extra                    428
mta_tax                  428
tip_amount               428
tolls_amount             428
improvement_surcharge    428
total_amount             428
payment_type             428
trip_type                428
congestion_surcharge     428
dtype: int64

In [1371]:
df_clean[(df_clean['DOLocationID']==264) & (df_clean['PULocationID']==264)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
115,2,2023-01-01 01:52:58,2023-01-01 01:53:00,N,1.0,264,264,1.0,0.00,3.0,1.0,0.5,21.5,0.0,0.3,26.3,1.0,1.0,0.0
245,2,2023-01-01 03:16:39,2023-01-01 03:16:42,N,5.0,264,264,1.0,0.00,10.0,0.0,0.0,0.0,0.0,1.0,11.0,2.0,2.0,0.0
283,2,2023-01-01 05:29:35,2023-01-01 05:29:37,N,5.0,264,264,1.0,0.00,12.0,0.0,0.0,2.6,0.0,1.0,15.6,1.0,2.0,0.0
1331,2,2023-01-02 04:41:06,2023-01-02 04:41:08,N,1.0,264,264,1.0,0.00,3.0,1.0,0.5,0.0,0.0,1.0,5.5,2.0,1.0,0.0
1581,2,2023-01-02 10:29:51,2023-01-02 10:30:34,N,1.0,264,264,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61912,1,2023-01-31 08:16:40,2023-01-31 08:45:26,N,1.0,264,264,1.0,0.00,19.5,0.0,1.5,0.0,0.0,1.0,21.0,1.0,1.0,0.0
62261,2,2023-01-31 11:00:15,2023-01-31 11:00:17,N,5.0,264,264,1.0,0.18,10.0,0.0,0.0,0.0,0.0,1.0,11.0,1.0,2.0,0.0
63725,2,2023-01-31 21:24:02,2023-01-31 21:24:03,N,5.0,264,264,1.0,0.00,35.0,0.0,0.0,0.0,0.0,1.0,36.0,1.0,2.0,0.0
63868,2,2023-01-31 23:44:44,2023-01-31 23:45:15,N,5.0,264,264,1.0,0.00,-10.0,0.0,0.0,0.0,0.0,-1.0,-11.0,4.0,2.0,0.0


In [1372]:
df_clean[(df_clean['DOLocationID']==264) | (df_clean['PULocationID']==264)].drop(index=df_clean[(df_clean['DOLocationID']==264) & (df_clean['PULocationID']==264)].index).count()

VendorID                 297
lpep_pickup_datetime     297
lpep_dropoff_datetime    297
store_and_fwd_flag       297
RatecodeID               297
PULocationID             297
DOLocationID             297
passenger_count          297
trip_distance            297
fare_amount              297
extra                    297
mta_tax                  297
tip_amount               297
tolls_amount             297
improvement_surcharge    297
total_amount             297
payment_type             297
trip_type                297
congestion_surcharge     297
dtype: int64

In [1373]:
df_clean[(df_clean['DOLocationID']==264) | (df_clean['PULocationID']==264)].drop(index=df_clean[(df_clean['DOLocationID']==264) & (df_clean['PULocationID']==264)].index)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
166,2,2023-01-01 02:37:12,2023-01-01 02:37:15,N,5.0,51,264,3.0,0.00,40.0,0.0,0.0,0.00,0.0,1.0,41.00,1.0,2.0,0.00
173,2,2023-01-01 02:37:12,2023-01-01 02:37:15,N,5.0,260,264,1.0,0.00,20.0,0.0,0.0,4.20,0.0,1.0,25.20,1.0,2.0,0.00
186,2,2023-01-01 03:01:54,2023-01-01 03:01:56,N,5.0,129,264,1.0,0.00,12.0,0.0,0.0,0.00,0.0,1.0,13.00,1.0,2.0,0.00
227,2,2023-01-01 03:11:48,2023-01-01 03:12:31,N,5.0,7,264,1.0,0.00,12.0,0.0,0.0,2.60,0.0,1.0,15.60,1.0,2.0,0.00
268,2,2023-01-01 03:50:22,2023-01-01 03:50:25,N,5.0,171,264,1.0,0.00,20.0,0.0,0.0,4.20,0.0,1.0,25.20,1.0,2.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61827,2,2023-01-31 07:40:03,2023-01-31 07:51:02,N,1.0,75,264,4.0,1.08,7.9,0.0,0.5,0.00,0.0,1.0,9.40,2.0,1.0,0.00
62730,2,2023-01-31 14:26:53,2023-01-31 14:46:12,N,1.0,75,264,1.0,1.69,15.6,0.0,0.5,1.98,0.0,1.0,21.83,1.0,1.0,2.75
63149,2,2023-01-31 17:25:41,2023-01-31 17:34:11,N,1.0,75,264,1.0,1.30,8.6,2.5,0.5,2.52,0.0,1.0,15.12,1.0,1.0,0.00
63756,2,2023-01-31 21:56:14,2023-01-31 21:56:16,N,5.0,97,264,1.0,0.00,12.0,0.0,0.0,0.00,0.0,1.0,13.00,1.0,2.0,0.00


In [1374]:
df_clean.drop(index=df_clean[(df_clean['DOLocationID']==264) | (df_clean['PULocationID']==264)].index, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63446 entries, 0 to 63886
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               63446 non-null  int64         
 1   lpep_pickup_datetime   63446 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  63446 non-null  datetime64[ns]
 3   store_and_fwd_flag     63446 non-null  object        
 4   RatecodeID             63446 non-null  float64       
 5   PULocationID           63446 non-null  int64         
 6   DOLocationID           63446 non-null  int64         
 7   passenger_count        63446 non-null  float64       
 8   trip_distance          63446 non-null  float64       
 9   fare_amount            63446 non-null  float64       
 10  extra                  63446 non-null  float64       
 11  mta_tax                63446 non-null  float64       
 12  tip_amount             63446 non-null  float64       
 13  tolls_

In [1375]:
df_clean.reset_index(inplace=True, drop=True)

In [1376]:
df_clean.tail()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
63441,2,2023-01-31 23:09:22,2023-01-31 23:17:21,N,5.0,130,205,1.0,2.75,15.0,0.0,0.0,2.00,0.0,0.3,17.30,1.0,2.0,0.00
63442,2,2023-01-31 23:06:18,2023-01-31 23:17:03,N,1.0,65,181,1.0,2.44,13.5,1.0,0.5,3.20,0.0,1.0,19.20,1.0,1.0,0.00
63443,2,2023-01-31 23:17:03,2023-01-31 23:23:04,N,1.0,244,116,1.0,1.40,9.3,1.0,0.5,2.36,0.0,1.0,14.16,1.0,1.0,0.00
63444,2,2023-01-31 23:29:18,2023-01-31 23:38:50,N,1.0,74,238,1.0,2.47,13.5,1.0,0.5,3.75,0.0,1.0,22.50,1.0,1.0,2.75
63445,2,2023-01-31 23:00:12,2023-01-31 23:10:08,N,1.0,95,95,1.0,1.39,11.4,1.0,0.5,4.17,0.0,1.0,18.07,1.0,1.0,0.00


#### Negative Surcharges

In [1377]:
df_clean["improvement_surcharge"].unique()

array([ 1. ,  0. ,  0.3, -1. , -0.3])

In [1378]:
df_clean[df_clean["improvement_surcharge"]==-1]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
650,2,2023-01-01 14:58:13,2023-01-01 14:58:39,N,1.0,75,74,2.0,0.17,-3.0,0.0,-0.5,0.0,0.0,-1.0,-4.5,3.0,1.0,0.0
884,2,2023-01-01 17:24:52,2023-01-01 17:25:12,N,1.0,193,193,1.0,0.00,-3.0,0.0,-0.5,0.0,0.0,-1.0,-4.5,4.0,1.0,0.0
1297,2,2023-01-02 02:05:08,2023-01-02 02:12:57,N,1.0,129,82,1.0,0.64,-8.6,-1.0,-0.5,0.0,0.0,-1.0,-11.1,3.0,1.0,0.0
1840,2,2023-01-02 13:51:33,2023-01-02 13:52:42,N,1.0,75,75,2.0,0.08,-3.7,0.0,-0.5,0.0,0.0,-1.0,-5.2,3.0,1.0,0.0
1895,2,2023-01-02 14:03:10,2023-01-02 14:03:26,N,1.0,75,75,1.0,0.00,-3.0,0.0,-0.5,0.0,0.0,-1.0,-4.5,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62775,2,2023-01-31 17:15:42,2023-01-31 17:16:13,N,1.0,193,193,1.0,0.00,-3.0,-2.5,-0.5,0.0,0.0,-1.0,-7.0,3.0,1.0,0.0
62804,2,2023-01-31 18:11:43,2023-01-31 18:12:13,N,1.0,193,193,1.0,0.00,-3.0,-2.5,-0.5,0.0,0.0,-1.0,-7.0,3.0,1.0,0.0
62922,2,2023-01-31 18:39:16,2023-01-31 18:39:29,N,1.0,130,130,3.0,0.05,-3.0,-2.5,-0.5,0.0,0.0,-1.0,-7.0,4.0,1.0,0.0
63108,2,2023-01-31 19:06:18,2023-01-31 19:06:46,N,1.0,75,75,2.0,0.08,-3.0,-2.5,-0.5,0.0,0.0,-1.0,-7.0,3.0,1.0,0.0


In [1379]:
df_clean["total_amount"][df_clean["improvement_surcharge"]==-1].unique()

array([ -4.5 , -11.1 ,  -5.2 ,  -5.4 ,  -5.9 , -11.5 ,  -8.4 ,  -7.  ,
       -11.2 ,  -7.7 ,  -5.5 ,  -8.  ,  -7.3 ,  -4.51, -13.3 ,  -5.33,
       -10.5 , -14.  , -61.  ,  -7.6 , -71.5 , -15.  , -10.1 ,  -9.4 ,
        -6.9 ,  -5.21, -12.5 , -21.  ,  -6.2 ,  -9.8 , -10.4 , -22.  ,
        -8.3 ,  -1.5 , -14.65,  -8.65,  -4.25,  -9.  , -51.  ,  -6.6 ,
        -8.7 , -18.2 ])

In [1380]:
df_clean["VendorID"][df_clean["improvement_surcharge"] < 0].unique()

array([2])

In [1381]:
df_clean["VendorID"][df_clean["improvement_surcharge"] >= 0].unique()

array([2, 1])

In [1382]:
df_clean["VendorID"][df_clean["improvement_surcharge"] < 0].count()

np.int64(168)

In [1383]:
df_clean["VendorID"][(df_clean["improvement_surcharge"] >= 0) & (df_clean["VendorID"] == 2)].count()

np.int64(54153)

In [1384]:
df_clean_plus_imp = df_clean[df_clean["improvement_surcharge"] >= 0].head()
df_clean_plus_imp.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,1.0,8.00,1.0,1.0,0.00


In [1385]:
df_clean_minus_imp = df_clean[df_clean["improvement_surcharge"] < 0]
df_clean_minus_imp.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
650,2,2023-01-01 14:58:13,2023-01-01 14:58:39,N,1.0,75,74,2.0,0.17,-3.0,0.0,-0.5,0.0,0.0,-1.0,-4.5,3.0,1.0,0.0
884,2,2023-01-01 17:24:52,2023-01-01 17:25:12,N,1.0,193,193,1.0,0.00,-3.0,0.0,-0.5,0.0,0.0,-1.0,-4.5,4.0,1.0,0.0
1297,2,2023-01-02 02:05:08,2023-01-02 02:12:57,N,1.0,129,82,1.0,0.64,-8.6,-1.0,-0.5,0.0,0.0,-1.0,-11.1,3.0,1.0,0.0
1663,2,2023-01-02 12:56:53,2023-01-02 13:00:10,N,1.0,75,75,1.0,0.77,-4.5,0.0,-0.5,0.0,0.0,-0.3,-5.3,4.0,1.0,0.0
1840,2,2023-01-02 13:51:33,2023-01-02 13:52:42,N,1.0,75,75,2.0,0.08,-3.7,0.0,-0.5,0.0,0.0,-1.0,-5.2,3.0,1.0,0.0


In [1386]:
list_item = []
for col in df_clean_plus_imp.columns:
    list_item.append([col, df_clean_plus_imp[col].nunique(), df_clean_plus_imp[col].unique()])

pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)

,Column Name,Number of Unique,Unique Sample
0,VendorID,2,"[2, 1]"
1,lpep_pickup_datetime,5,"[2023-01-01 00:26:10, 2023-01-01 00:51:03, 2023-01-01 00:35:12, 2023-01-01 00:13:14, 2023-01-01 00:33:04]"
2,lpep_dropoff_datetime,5,"[2023-01-01 00:37:11, 2023-01-01 00:57:49, 2023-01-01 00:41:32, 2023-01-01 00:19:03, 2023-01-01 00:39:02]"
3,store_and_fwd_flag,1,[N]
4,RatecodeID,1,[1.0]
5,PULocationID,4,"[166, 24, 223, 41]"
6,DOLocationID,5,"[143, 43, 179, 238, 74]"
7,passenger_count,1,[1.0]
8,trip_distance,5,"[2.58, 1.81, 0.0, 1.3, 1.1]"
9,fare_amount,5,"[14.9, 10.7, 7.2, 6.5, 6.0]"


In [1387]:
list_item = []
for col in df_clean_minus_imp.columns:
    list_item.append([col, df_clean_minus_imp[col].nunique(), df_clean_minus_imp[col].unique()])

pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)

,Column Name,Number of Unique,Unique Sample
0,VendorID,1,[2]
1,lpep_pickup_datetime,168,"[2023-01-01 14:58:13, 2023-01-01 17:24:52, 2023-01-02 02:05:08, 2023-01-02 12:56:53, 2023-01-02 13:51:33, 2023-01-02 14:03:10, 2023-01-02 19:03:48, 2023-01-03 10:18:45, 2023-01-03 11:30:37, 2023-01-03 11:41:06, 2023-01-03 11:43:10, 2023-01-03 11:45:06, 2023-01-03 12:46:39, 2023-01-03 13:53:25, 2023-01-03 14:25:22, 2023-01-03 13:59:49, 2023-01-03 15:51:13, 2023-01-04 06:46:52, 2023-01-04 09:36:20, 2023-01-04 14:26:41, 2023-01-04 16:13:03, 2023-01-04 17:38:20, 2023-01-04 17:06:10, 2023-01-04 1..."
2,lpep_dropoff_datetime,168,"[2023-01-01 14:58:39, 2023-01-01 17:25:12, 2023-01-02 02:12:57, 2023-01-02 13:00:10, 2023-01-02 13:52:42, 2023-01-02 14:03:26, 2023-01-02 19:04:24, 2023-01-03 10:20:36, 2023-01-03 11:32:23, 2023-01-03 11:41:54, 2023-01-03 11:43:25, 2023-01-03 11:45:27, 2023-01-03 12:46:57, 2023-01-03 13:55:03, 2023-01-03 14:27:53, 2023-01-03 14:00:46, 2023-01-03 16:00:58, 2023-01-04 06:47:27, 2023-01-04 09:36:25, 2023-01-04 14:26:52, 2023-01-04 16:15:30, 2023-01-04 17:39:14, 2023-01-04 17:11:35, 2023-01-04 1..."
3,store_and_fwd_flag,1,[N]
4,RatecodeID,3,"[1.0, 5.0, 2.0]"
5,PULocationID,35,"[75, 193, 129, 130, 260, 166, 41, 42, 7, 97, 74, 95, 134, 83, 181, 24, 179, 236, 82, 61, 65, 255, 256, 36, 80, 235, 78, 43, 55, 131, 112, 66, 116, 70, 248]"
6,DOLocationID,37,"[74, 193, 82, 75, 130, 260, 166, 41, 42, 7, 97, 95, 134, 83, 181, 238, 196, 189, 129, 61, 66, 255, 179, 256, 36, 80, 78, 236, 55, 131, 23, 24, 112, 116, 43, 70, 248]"
7,passenger_count,6,"[2.0, 1.0, 4.0, 6.0, 5.0, 3.0]"
8,trip_distance,55,"[0.17, 0.0, 0.64, 0.77, 0.08, 0.11, 0.16, 0.06, 0.32, 0.59, 0.01, 0.22, 0.1, 0.75, 0.04, 0.05, 0.93, 0.84, 0.44, 0.71, 0.13, 0.61, 0.54, 0.03, 0.18, 0.25, 0.14, 0.88, 0.82, 0.29, 0.23, 0.07, 0.72, 0.41, 0.09, 0.26, 0.19, 0.66, 0.02, 0.74, 0.4, 0.15, 0.99, 2.34, 0.31, 0.6, 0.36, 24.14, 0.52, 0.62, 0.87, 0.39, 0.24, 0.37, 0.67]"
9,fare_amount,23,"[-3.0, -8.6, -4.5, -3.7, -4.4, -10.0, -7.2, -2.5, -6.5, -4.0, -5.8, -9.3, -13.0, -60.0, -5.1, -70.0, -7.9, -20.0, -21.0, 0.0, -50.0, -15.0, -14.2]"


In [1388]:
for i in range(len(df_clean)):
    if df_clean["improvement_surcharge"][i] < 0:
        df_clean["fare_amount"][i] = abs(df_clean["fare_amount"][i])
        df_clean["extra"][i] = abs(df_clean["extra"][i])
        df_clean["mta_tax"][i] = abs(df_clean["mta_tax"][i])
        df_clean["tip_amount"][i] = abs(df_clean["tip_amount"][i])
        df_clean["improvement_surcharge"][i] = abs(df_clean["improvement_surcharge"][i])
        df_clean["total_amount"][i] = abs(df_clean["total_amount"][i])
        df_clean["congestion_surcharge"][i] = abs(df_clean["congestion_surcharge"][i])


In [1389]:
df_clean["improvement_surcharge"].unique()

array([1. , 0. , 0.3])

In [1390]:
df_clean["fare_amount"][df["fare_amount"] < 0].unique()

array([ 10. ,   3.7,  41.5,  19.1,  19.8,  10.5,  22.6,   5.8,  13.5,
        11.4,   7.2,  39.4,  21.2,  16.3,  12.8,   7.9,   9.3,  15.6,
         8.6,  26.1,  10.7,   0. ,  17.7,  59. ,  20.5,  18.4,  14.9,
        14.2,  21.9,  24. ,  31. ,  58.3,  62.5,   9.5, 103.1, 200. ,
        17. ,   7. ,  12.1,  44.3,  36.6,   6.5,   4. ,  33.8,  27.5,
        70. ,  11.5,  29.6,   3. ,  23.3,  33.1,  35.2,  30.3,  15. ,
        19. ,  18.2])

### Re-check Duplicates

In [1391]:
df_clean.duplicated().sum()

np.int64(168)

In [1392]:
df_clean.drop_duplicates(inplace=True)

### Re-Check Unique Values in Columns

In [1393]:
list_item = []
for col in df_clean.columns:
    list_item.append([col, df_clean[col].nunique(), df_clean[col].unique()])

pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)

,Column Name,Number of Unique,Unique Sample
0,VendorID,2,"[2, 1]"
1,lpep_pickup_datetime,62257,"[2023-01-01 00:26:10, 2023-01-01 00:51:03, 2023-01-01 00:35:12, 2023-01-01 00:13:14, 2023-01-01 00:33:04, 2023-01-01 00:53:31, 2023-01-01 00:09:14, 2023-01-01 00:11:58, 2023-01-01 00:41:29, 2023-01-01 00:50:32, 2023-01-01 00:16:12, 2023-01-01 00:08:43, 2023-01-01 00:26:32, 2023-01-01 00:18:35, 2023-01-01 00:39:32, 2023-01-01 00:49:34, 2023-01-01 00:10:45, 2023-01-01 00:35:11, 2023-01-01 00:31:06, 2023-01-01 00:14:37, 2023-01-01 00:26:21, 2023-01-01 00:41:56, 2023-01-01 00:32:02, 2023-01-01 0..."
2,lpep_dropoff_datetime,62188,"[2023-01-01 00:37:11, 2023-01-01 00:57:49, 2023-01-01 00:41:32, 2023-01-01 00:19:03, 2023-01-01 00:39:02, 2023-01-01 01:11:04, 2023-01-01 00:26:39, 2023-01-01 00:24:55, 2023-01-01 00:46:26, 2023-01-01 01:13:42, 2023-01-01 00:41:43, 2023-01-01 00:17:08, 2023-01-01 00:45:31, 2023-01-01 00:30:09, 2023-01-01 01:18:06, 2023-01-01 01:08:23, 2023-01-01 00:19:37, 2023-01-01 01:17:35, 2023-01-01 00:42:23, 2023-01-01 00:21:50, 2023-01-01 00:30:26, 2023-01-01 00:46:06, 2023-01-01 00:41:25, 2023-01-01 0..."
3,store_and_fwd_flag,2,"[N, Y]"
4,RatecodeID,5,"[1.0, 5.0, 4.0, 3.0, 2.0]"
5,PULocationID,219,"[166, 24, 223, 41, 181, 255, 75, 66, 195, 83, 97, 7, 42, 95, 260, 134, 130, 258, 129, 116, 92, 65, 74, 17, 152, 210, 112, 69, 243, 82, 179, 168, 80, 25, 145, 33, 226, 52, 169, 36, 157, 43, 256, 173, 70, 251, 40, 101, 61, 241, 146, 147, 228, 56, 244, 263, 196, 37, 236, 213, 28, 117, 14, 188, 171, 177, 29, 262, 250, 72, 216, 22, 20, 55, 257, 193, 127, 219, 3, 119, 265, 167, 78, 135, 247, 93, 159, 48, 31, 91, 53, 160, 94, 47, 254, 215, 10, 49, 197, 9, ...]"
6,DOLocationID,247,"[143, 43, 179, 238, 74, 262, 45, 75, 166, 140, 234, 148, 255, 186, 210, 129, 68, 260, 42, 24, 244, 146, 135, 173, 216, 142, 265, 164, 7, 83, 95, 130, 223, 80, 48, 263, 196, 250, 226, 239, 189, 41, 168, 162, 141, 33, 145, 151, 116, 195, 249, 235, 160, 98, 79, 229, 181, 193, 237, 233, 102, 10, 37, 258, 82, 170, 61, 112, 231, 97, 17, 169, 96, 219, 243, 236, 56, 36, 114, 157, 161, 54, 90, 40, 188, 70, 63, 251, 101, 92, 153, 53, 57, 256, 65, 257, 138, 85, 52, 119, ...]"
7,passenger_count,10,"[1.0, 2.0, 4.0, 3.0, 6.0, 5.0, 0.0, 9.0, 8.0, 7.0]"
8,trip_distance,1721,"[2.58, 1.81, 0.0, 1.3, 1.1, 2.78, 3.8, 1.88, 1.11, 4.22, 4.8, 1.99, 7.08, 3.23, 5.91, 11.3, 8.28, 2.02, 0.96, 0.53, 1.51, 0.88, 2.48, 3.53, 2.25, 4.29, 2.53, 1.7, 10.44, 6.78, 1.39, 1.31, 6.7, 3.52, 1.74, 2.31, 1.37, 10.35, 1.05, 6.19, 2.8, 1.13, 3.19, 1.26, 2.87, 8.55, 4.46, 1.18, 2.5, 9.01, 0.63, 2.08, 1.47, 7.83, 2.15, 12.73, 1.62, 7.3, 2.73, 4.9, 2.17, 0.6, 0.8, 2.39, 1.23, 10.45, 2.23, 2.21, 5.2, 5.25, 6.32, 3.82, 3.58, 0.7, 2.91, 5.13, 12.4, 1.45, 7.52, 2.45, 0.54, 0.47, 1.16, 15.39, 5..."
9,fare_amount,430,"[14.9, 10.7, 7.2, 6.5, 6.0, 17.7, 19.1, 14.2, 24.7, 26.8, 11.4, 30.3, 35.9, 32.5, 15.0, 44.3, 12.8, 8.6, 5.8, 16.3, 19.8, 18.4, 13.5, 8.0, 47.1, 60.0, 7.9, 10.0, 3.0, 5.5, 17.0, 75.0, 15.6, 47.8, 21.2, 37.3, 52.0, 4.4, 51.3, 25.5, 5.1, 9.3, 29.6, 48.5, 28.2, 21.9, 5.0, 4.0, 25.4, 36.6, 66.0, 20.0, 50.0, 190.0, 20.5, 46.4, 12.1, 31.0, 33.8, 26.1, 22.6, 15.5, 24.0, 25.0, 10.5, 19.5, 100.0, 40.0, 31.7, 40.8, 80.0, 110.0, 14.5, 43.6, 23.0, 90.0, 49.2, 16.0, 28.9, 30.0, 25.8, 35.0, 35.2, 7.0, 78...."


### Penyimpanan Data Bersih

In [1394]:
df_clean.to_csv("df_clean.csv", index=False)

## **2.2. Data yang Sudah Bersih**

In [1395]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63278 entries, 0 to 63445
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               63278 non-null  int64         
 1   lpep_pickup_datetime   63278 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  63278 non-null  datetime64[ns]
 3   store_and_fwd_flag     63278 non-null  object        
 4   RatecodeID             63278 non-null  float64       
 5   PULocationID           63278 non-null  int64         
 6   DOLocationID           63278 non-null  int64         
 7   passenger_count        63278 non-null  float64       
 8   trip_distance          63278 non-null  float64       
 9   fare_amount            63278 non-null  float64       
 10  extra                  63278 non-null  float64       
 11  mta_tax                63278 non-null  float64       
 12  tip_amount             63278 non-null  float64       
 13  tolls_

## **2.3. Data Preparation**

In [1396]:
taxizone = pd.read_csv("taxi_zone_lookup.csv")

In [1397]:
taxizone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [1398]:
df_clean.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,1.0,8.00,1.0,1.0,0.00


In [1399]:
taxizone_PU = taxizone.copy()
taxizone_PU.rename(columns={"LocationID": "PULocationID", "Borough": "PUBorough", "Zone": "PUZone", "service_zone": "PUservice_zone"}, inplace=True)
taxizone_PU.head()

,PULocationID,PUBorough,PUZone,PUservice_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [1400]:
taxizone_DO = taxizone.copy()
taxizone_DO.rename(columns={"LocationID": "DOLocationID", "Borough": "DOBorough", "Zone": "DOZone", "service_zone": "DOservice_zone"}, inplace=True)
taxizone_DO.head()

,DOLocationID,DOBorough,DOZone,DOservice_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [1401]:
df_clean = df_clean.merge(taxizone_PU, how="left", on="PULocationID")
df_clean = df_clean.merge(taxizone_DO, how="left", on="DOLocationID")
df_clean

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,total_amount,payment_type,trip_type,congestion_surcharge,PUBorough,PUZone,PUservice_zone,DOBorough,DOZone,DOservice_zone
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,...,24.18,1.0,1.0,2.75,Manhattan,Morningside Heights,Boro Zone,Manhattan,Lincoln Square West,Yellow Zone
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,...,15.84,1.0,1.0,0.00,Manhattan,Bloomingdale,Yellow Zone,Manhattan,Central Park,Yellow Zone
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,...,11.64,1.0,1.0,0.00,Queens,Steinway,Boro Zone,Queens,Old Astoria,Boro Zone
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,...,10.20,1.0,1.0,0.00,Manhattan,Central Harlem,Boro Zone,Manhattan,Upper West Side North,Yellow Zone
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,...,8.00,1.0,1.0,0.00,Manhattan,Central Harlem,Boro Zone,Manhattan,East Harlem North,Boro Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63273,2,2023-01-31 23:09:22,2023-01-31 23:17:21,N,5.0,130,205,1.0,2.75,15.0,...,17.30,1.0,2.0,0.00,Queens,Jamaica,Boro Zone,Queens,Saint Albans,Boro Zone
63274,2,2023-01-31 23:06:18,2023-01-31 23:17:03,N,1.0,65,181,1.0,2.44,13.5,...,19.20,1.0,1.0,0.00,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Park Slope,Boro Zone
63275,2,2023-01-31 23:17:03,2023-01-31 23:23:04,N,1.0,244,116,1.0,1.40,9.3,...,14.16,1.0,1.0,0.00,Manhattan,Washington Heights South,Boro Zone,Manhattan,Hamilton Heights,Boro Zone
63276,2,2023-01-31 23:29:18,2023-01-31 23:38:50,N,1.0,74,238,1.0,2.47,13.5,...,22.50,1.0,1.0,2.75,Manhattan,East Harlem North,Boro Zone,Manhattan,Upper West Side North,Yellow Zone


In [1402]:
df_clean[df_clean["PUBorough"] != df_clean["DOBorough"]]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,total_amount,payment_type,trip_type,congestion_surcharge,PUBorough,PUZone,PUservice_zone,DOBorough,DOZone,DOservice_zone
6,1,2023-01-01 00:09:14,2023-01-01 00:26:39,N,1.0,181,45,2.0,3.80,19.1,...,29.20,1.0,1.0,2.75,Brooklyn,Park Slope,Boro Zone,Manhattan,Chinatown,Yellow Zone
10,1,2023-01-01 00:16:12,2023-01-01 00:41:43,N,1.0,255,234,2.0,4.80,26.8,...,32.05,1.0,1.0,2.75,Brooklyn,Williamsburg (North Side),Boro Zone,Manhattan,Union Sq,Yellow Zone
14,2,2023-01-01 00:39:32,2023-01-01 01:18:06,N,1.0,255,186,1.0,5.91,35.9,...,41.15,2.0,1.0,2.75,Brooklyn,Williamsburg (North Side),Boro Zone,Manhattan,Penn Station/Madison Sq West,Yellow Zone
17,2,2023-01-01 00:35:11,2023-01-01 01:17:35,N,1.0,97,68,1.0,8.28,44.3,...,54.55,1.0,1.0,2.75,Brooklyn,Fort Greene,Boro Zone,Manhattan,East Chelsea,Yellow Zone
25,2,2023-01-01 00:22:31,2023-01-01 00:39:24,N,1.0,255,146,1.0,3.53,19.8,...,26.76,1.0,1.0,0.00,Brooklyn,Williamsburg (North Side),Boro Zone,Queens,Long Island City/Queens Plaza,Boro Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63195,2,2023-01-31 22:09:40,2023-01-31 22:41:01,N,1.0,66,162,2.0,8.41,40.8,...,57.56,1.0,1.0,2.75,Brooklyn,DUMBO/Vinegar Hill,Boro Zone,Manhattan,Midtown East,Yellow Zone
63201,2,2023-01-31 22:06:34,2023-01-31 22:18:44,N,1.0,256,232,2.0,2.50,14.9,...,25.19,1.0,1.0,2.75,Brooklyn,Williamsburg (South Side),Boro Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone
63220,2,2023-01-31 22:28:16,2023-01-31 22:39:45,N,1.0,65,231,1.0,2.59,14.9,...,24.18,1.0,1.0,2.75,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Manhattan,TriBeCa/Civic Center,Yellow Zone
63235,2,2023-01-31 22:49:39,2023-01-31 23:02:07,N,1.0,74,212,1.0,4.30,20.5,...,23.00,2.0,1.0,0.00,Manhattan,East Harlem North,Boro Zone,Bronx,Soundview/Bruckner,Boro Zone


In [1403]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63278 entries, 0 to 63277
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               63278 non-null  int64         
 1   lpep_pickup_datetime   63278 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  63278 non-null  datetime64[ns]
 3   store_and_fwd_flag     63278 non-null  object        
 4   RatecodeID             63278 non-null  float64       
 5   PULocationID           63278 non-null  int64         
 6   DOLocationID           63278 non-null  int64         
 7   passenger_count        63278 non-null  float64       
 8   trip_distance          63278 non-null  float64       
 9   fare_amount            63278 non-null  float64       
 10  extra                  63278 non-null  float64       
 11  mta_tax                63278 non-null  float64       
 12  tip_amount             63278 non-null  float64       
 13  t

In [1404]:
df_clean["Vendor"] = df_clean["VendorID"].copy()

for i in range(len(df_clean["Vendor"])):
    if df_clean["Vendor"][i] == 1:
        df_clean["Vendor"][i] = "CreativeMobileTechnologies,LLC."
    elif df_clean["Vendor"][i] == 2:
        df_clean["Vendor"][i] = "VeriFoneInc."

df_clean[["VendorID", "Vendor"]].head(10)

,VendorID,Vendor
0,2,VeriFoneInc.
1,2,VeriFoneInc.
2,2,VeriFoneInc.
3,1,"CreativeMobileTechnologies,LLC."
4,1,"CreativeMobileTechnologies,LLC."
5,2,VeriFoneInc.
6,1,"CreativeMobileTechnologies,LLC."
7,2,VeriFoneInc.
8,2,VeriFoneInc.
9,2,VeriFoneInc.


In [1405]:
df_clean["Ratecode"] = df_clean["RatecodeID"].copy()

for i in range(len(df_clean["Ratecode"])):
    if df_clean["Ratecode"][i] == 1:
        df_clean["Ratecode"][i] = "Standard rate"
    elif df_clean["Ratecode"][i] == 2:
        df_clean["Ratecode"][i] = "JFK"
    elif df_clean["Ratecode"][i] == 3:
        df_clean["Ratecode"][i] = "Newark"
    elif df_clean["Ratecode"][i] == 4:
        df_clean["Ratecode"][i] = "Nassau or Westchester"
    elif df_clean["Ratecode"][i] == 5: 
        df_clean["Ratecode"][i] = "Negotiated fare"
    elif df_clean["Ratecode"][i] == 6:
        df_clean["Ratecode"][i] = "Group ride"

df_clean[["RatecodeID", "Ratecode"]].head(10)

,RatecodeID,Ratecode
0,1.0,Standard rate
1,1.0,Standard rate
2,1.0,Standard rate
3,1.0,Standard rate
4,1.0,Standard rate
5,1.0,Standard rate
6,1.0,Standard rate
7,1.0,Standard rate
8,1.0,Standard rate
9,1.0,Standard rate


In [1406]:
df_clean["payment"] = df_clean["payment_type"].copy()

for i in range(len(df_clean["payment"])):
    if df_clean["payment"][i] == 1:
        df_clean["payment"][i] = "Credit card"
    elif df_clean["payment"][i] == 2:
        df_clean["payment"][i] = "Cash"
    elif df_clean["payment"][i] == 3:
        df_clean["payment"][i] = "No charge"
    elif df_clean["payment"][i] == 4:
        df_clean["payment"][i] = "Dispute"
    elif df_clean["payment"][i] == 5: 
        df_clean["payment"][i] = "Unknown"
    elif df_clean["payment"][i] == 6:
        df_clean["payment"][i] = "Voided trip"

df_clean[["payment_type", "payment"]].head(10)

,payment_type,payment
0,1.0,Credit card
1,1.0,Credit card
2,1.0,Credit card
3,1.0,Credit card
4,1.0,Credit card
5,2.0,Cash
6,1.0,Credit card
7,2.0,Cash
8,1.0,Credit card
9,1.0,Credit card


In [1407]:
df_clean["trip"] = df_clean["trip_type"].copy()

for i in range(len(df_clean["trip"])):
    if df_clean["trip"][i] == 1:
        df_clean["trip"][i] = "Street-hail"
    elif df_clean["trip"][i] == 2:
        df_clean["trip"][i] = "Dispatch"

df_clean[["trip_type", "trip"]].head(10)

,trip_type,trip
0,1.0,Street-hail
1,1.0,Street-hail
2,1.0,Street-hail
3,1.0,Street-hail
4,1.0,Street-hail
5,1.0,Street-hail
6,1.0,Street-hail
7,1.0,Street-hail
8,1.0,Street-hail
9,1.0,Street-hail


# **3. Data Analysis**

# **4. Kesimpulan dan Rekomendasi** 
